# Scenario 5: Extracting LAI Timeseries from Sentinel-2 imagery
This notebook is complementary material for the walkthrough scenario **Extracting LAI Timeseries from Sentinel-2 imagery** used the STELAR KLMS
It is not intended to be run as a standalone notebook. It **requires access to a deployment of STELAR KLMS** and an **account** on the respective instance. 

Some of the instances used during the evaluation period of the STELAR Project are:

Internal Pilot Instance: https://klms.stelar.gr

Public Sandbox Instance: https://sandbox.stelar.gr


*If you don't have an account on the STELAR KLMS, you can create one on the respective instance. 
Kindly note that the internal pilot instance is only accessible to STELAR project members, while the public sandbox instance is open to everyone by registration.*

---
# Overview

This notebook is intended to run **TS-Extract** tool to extract the pixel-level Leaf Area Index (LAI) timeseries from Sentinel-2 imagery, after segmenting the crop fields in the Austrian North region.

### Prerequisites

- Fill in your accounts credentials in the block below. 
- Select datasets according to the walkthrough directions.
- Ensure you have a modern python version installed (3.9 or later).
- Install the STELAR Python SDK and any other required libraries (`pip install stelar_client --upgrade`).

### Instatiate a STELAR Client object
**Modify credentials and base URL as needed.**

In [ ]:
from stelar.client import Client, Dataset, TaskSpec, Process
from datetime import datetime

# Base URL
# Sandbox: https://sandbox.stelar.gr
# Internal Pilots: https://klms.stelar.gr

BASE_URL = "https://sandbox.stelar.gr"
USERNAME = "your_username"  # Replace with your username
PASSWORD = "your_password"  # Replace with your password

c = Client(base_url=BASE_URL, username=USERNAME, password=PASSWORD)
# Normalize username to be used
username = c.GET("v1/users/me").json()['result']['preferred_username']
print(f"Connected to STELAR KLMS @ {c._base_url} as {username}")

### Select dataset containing the LAI single band images and a dataset containing the boundaries of the segmented fields.

In [ ]:
lai_imagery_dataset = c.datasets["tile-33uvp-lai-tifs-for-2021"]
segmented_fields_dataset = c.datasets["tile-33uvp-reflectance-sentinel-2-imagery-2021"]
print(f"Selected Dataset: {lai_imagery_dataset.id} | {lai_imagery_dataset.title}")
print(f"Browse the dataset at: {c._base_url}/console/v1/catalog/{lai_imagery_dataset.id}")

### Create/Select a Workflow Process to run the segmentation task

In [ ]:
ORGANIZATION = "stelar-klms"

try:
    proc = c.processes.create(**{
        "title": "Evalution Workflow for " + username,
        "name": "evaluation-workflow-" + username,
        "organization": c.organizations[ORGANIZATION]
    })
    print(f"Created new process for evaluation: {proc.id} | {proc.title}")
except Exception as e:
    proc = c.processes["evaluation-workflow-" + username]
    print(f"Using existing process for evaluation: {proc.id} | {proc.title}")

### Create a dataset to store the results of the TS-extraction task

In [ ]:
ORGANIZATION = "stelar-klms"

try:
    res_dset = c.datasets.create(**{
        "title": "LAI Timeseries 33UVP for " + username,
        "name": "lai-timeseries-" + username,
        "organization": c.organizations[ORGANIZATION],
        "notes": "LAI Timeseries of 33UVP curated by " + username,
    })
    print(f"Created new dataset for storing LAI timeseries: {res_dset.id} | {res_dset.title}")
except Exception as e:
    res_dset = c.datasets["lai-timeseries-" + username]
    print(f"Using existing dataset for storing LAI timeseries: {res_dset.id} | {res_dset.title}")

### Prepare & Run the TS-Extract task

In [ ]:
# Start building the TaskSpec for Field Segmentation
t = TaskSpec(tool="ts-extract", name="Timeseries Extraction on 33UVP for "+username)

# Define the local dataset aliases
t.d(alias='d0', dset=res_dset)

# Define the inputs / LAI Imagery dataset (all images in the dataset)
t.i(images=str(lai_imagery_dataset.id))

# Set the outputs
timestamp= datetime.now().strftime("%Y%m%d%H%M%S")

# Define the location for storing the pixel level LAI timeseries
t.o(pixel_timeseries={
    "url": f"s3://klms-bucket/evaluation/experiments/proc-" + str(proc.id) + f"/timeseries_{timestamp}_"+username+".csv",
    "resource":{
        "name": "Pixel Level LAI Timeseries for " + username,
        "description": "Pixel level timeseries extracted from segmented fields for 33UVP tile",
        "format": "csv",
        "relation": "pixel_timeseries"
    },
    "dataset": "d0",
})

# Run the task using the workflow process created earlier
fsegm_task = proc.run(t)
print(f"Task {fsegm_task.id} is running. Check the status at: {c._base_url}/console/v1/task/{str(proc.id)}/{str(fsegm_task.id)}")